In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objects as go

from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
import pickle

pd.set_option('display.max_rows', 500, 'display.max_columns', 500,
              'display.width', 1000)

### Read data

In [ ]:
filepath='resources/final_probs.csv'
df=pd.read_csv(filepath)
names=df['username'].values
index=df['username'].index.values
nameslist = list(zip(index, names))

In [ ]:
df.columns

In [ ]:
df[df['username'] == 'jennsbeautifuldestinations']['follower_probability']

### Batch inference

In [ ]:
# # Build App
# app = JupyterDash(__name__)
# app.layout = html.Div([
#     html.H3('Results for Testing Dataset'),
#     html.Div([
#         html.Div([
#             dcc.Dropdown(
#                 id='page-3-dropdown',
#                 options=[{'label': k, 'value': i} for i,k in nameslist],
#                 value=nameslist[0][0]
#         )
#         ]),
#         html.Div(id='page-3-content', style={'fontSize':18}),
#         html.Div(id='follower_probability', style={'fontSize':18, 'color':'red'}),
#         html.Table(id='follower-characteristics'),
#     ])
# ])

# # Define callback to update graph
# # Tab 3 callback # 1
# @app.callback(Output('page-3-content', 'children'),
#               [Input('page-3-dropdown', 'value')])
# def page_3_dropdown(value):
#     name=df.loc[value, 'username']
#     return f'You have selected "{name}"'

# # Tab 3 callback # 2
# @app.callback(Output('follower-characteristics', 'children'),
#               [Input('page-3-dropdown', 'value')])
# def page_3_characteristics(value):
#     mydata=df.drop(['actual', 'follower_probability', 'username'], axis=1)
#     mydata=df[['is_private', 'mediacount', 'followers', 'followees', 'is_business_account', 'has_public_story']]
#     return html.Table(
#         [html.Tr([html.Th(col) for col in mydata.columns])] +
#         [html.Tr([
#             html.Td(mydata.iloc[value][col]) for col in mydata.columns
#         ])]
#     )

# # Tab 3 callback # 2
# @app.callback(Output('follower_probability', 'children'),
#               [Input('page-3-dropdown', 'value')])

# def page_3_survival(value):
#     survival=df.loc[value, 'follower_probability']
#     actual = df.loc[value,'actual']
#     survival=round(survival*100)
#     return f'Predicted probability of survival is {survival}%, Actual survival is {actual}'


# # Run app and display result inline in the notebook
# app.run_server(mode='inline')

### Individual Predictions

In [ ]:
#! pip install dash-bootstrap-components

In [ ]:
# Dash_App.py
### Import Packages ########################################
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import numpy as np
import joblib
import pickle
### Setup ###################################################
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = 'Machine Learning Model Deployment'
server = app.server
# Load Preprocess
preprocess = joblib.load('resources/preprocess.joblib')
### load ML model ###########################################
with open('resources/final_model.pkl', 'rb') as f:
    model = pickle.load(f)
### App Layout ###############################################
app.layout = html.Div([
    dbc.Row([html.H4(children='Would the profile become a follower?')]),
    dbc.Row(
        [
            dbc.Col(html.Label(children='Media Count:'), width="auto"),
            dbc.Col(dcc.Input(id='mediacount',
                              type='number',
                              min=0,
                              max=1000,
                              step=1,
                              value=0),
                    width="auto")
        ],
        style={
            'width': '100%',
            'display': 'flex',
            'align-items': 'center',
            'justify-content': 'center'
        }),
    html.Br(),
    dbc.Row(
        [
            dbc.Col(html.Label(children='Followers:'), width="auto"),
            dbc.Col(dcc.Input(id='followers',
                              type='number',
                              min=0,
                              max=1000,
                              step=1,
                              value=0),
                    width="auto")
        ],
        style={
            'width': '100%',
            'display': 'flex',
            'align-items': 'center',
            'justify-content': 'center'
        }),
    html.Br(),
    dbc.Row(
        [
            dbc.Col(html.Label(children='Followees:'), width="auto"),
            dbc.Col(dcc.Input(id='followees',
                              type='number',
                              min=0,
                              max=1000,
                              step=1,
                              value=0),
                    width="auto")
        ],
        style={
            'width': '100%',
            'display': 'flex',
            'align-items': 'center',
            'justify-content': 'center'
        }),
    html.Br(),
    dbc.Row([
        dbc.Col(html.Label(children='Is Private:'), width="auto"),
        dbc.Col(dbc.RadioItems(
            id='is_private',
            options=[{
                'label': i,
                'value': i
            } for i in [0, 1]],
            value=0,
        ),
                width="auto"),
        dbc.Col(html.Label(children='Is Business Account:'), width="auto"),
        dbc.Col(dbc.RadioItems(
            id='is_business_account',
            options=[{
                'label': i,
                'value': i
            } for i in [0, 1]],
            value=0,
        ),
                width="auto"),
        dbc.Col(html.Label(children='Has Public Story:'), width="auto"),
        dbc.Col(dbc.RadioItems(
            id='has_public_story',
            options=[{
                'label': i,
                'value': i
            } for i in [0, 1]],
            value=1,
        ),
                width="auto")
    ]),
    html.Br(),
    dbc.Row([
        dbc.Button('Submit',
                   id='submit-val',
                   n_clicks=0,
                   color="primary",
                   className="mt-1",
                   style={
                       'text-align': 'center',
                       'fontSize': 18
                   })
    ]),
    html.Br(),
    dbc.Row([html.Div(id='prediction output')],
            style={
                'color': 'red',
                'text-align': 'center',
                'fontSize': 18
            })
],
                      className='nine columns',
                      style={'display': 'inline-block'})


### Callback to produce the prediction #########################
@app.callback(Output('prediction output', 'children'),
              Input('submit-val', 'n_clicks'), State('mediacount', 'value'),
              State('followers', 'value'), State('followees', 'value'),
              State('is_private', 'value'),
              State('is_business_account', 'value'),
              State('has_public_story', 'value'))
def update_output(n_clicks, mediacount, followers, followees, is_private,
                  is_business_account, has_public_story):
    inputs = [
        mediacount, followers, followees, is_private, is_business_account,
        has_public_story
    ]
    keys = [
        'mediacount', 'followers', 'followees', 'is_private',
        'is_business_account', 'has_public_story'
    ]
    dict6 = dict(zip(keys, inputs))
    df = pd.DataFrame([dict6])
    # drop unnecessary columns, and reorder columns to match the final model.

    df = df[[
        'is_private', 'mediacount', 'followers', 'followees',
        'is_business_account', 'has_public_story'
    ]]

    df_new = preprocess.transform(df)
    prob = model.predict_proba(df_new)
    final_prob = round(float(prob[0][1]) * 100, 1)
    return (f'Probability of Survival: {final_prob}%')


# Run app and display result inline in the notebook
app.run_server(mode='inline')